This is a short notebook to explore alternative ways of calculating curve length. Given a curve $\gamma : [0,1] \rightarrow \mathbb R^d$ we define it's length on an isotropic Riemannian manifold as
$$L[\gamma] = \int_0^1a(\gamma(t))\|\gamma'(t)\| dt.$$
Where $0 < \alpha_1 \leq a \leq \alpha_2$.
Suppose that our approximation of $\gamma$, denoted $\tilde \gamma$, is the piecewise linear interpolation of the points $\{\mathbf x_i\}_{i=0,...,N} \subset \mathbb R^d$. To define an explicit parameterisation fix $i \in \{1,...,N\}$. Then we have
$$\tilde \gamma(t) = N(\mathbf x_i - \mathbf x_{i-1})\left(t - \frac{i-1}{N}\right) + \mathbf x_{i-1} \text{ in } t \in \left[ \frac{i-1}{N}, \frac{i}{N} \right).$$
Consequently, we have, by splitting the integral
$$L[\tilde \gamma] = \sum_{i=1}^N \int_{(i-1)/N}^{i/N}a(\tilde \gamma(t))\|\tilde \gamma'(t)\| dt. $$
Using the explicit parameterisation for $\tilde \gamma$
$$\int_{(i-1)/N}^{i/N}a(\tilde \gamma(t))\|\tilde \gamma'(t)\| dt = \int_{(i-1)/N}^{i/N}a(\tilde \gamma(t))\|N(\mathbf x_i - \mathbf x_{i-1})\| dt \\= N\|\mathbf x_i - \mathbf x_{i-1}\|\int_{(i-1)/N}^{i/N}a(\tilde \gamma(t)) dt.$$
Denote the composite uniform trapezoidal rule as $T(M,x_0,x_1,f)$ where $1/M$ is the mesh size, $x_0, x_1$ are the limits of the integral and $f$ is the integrand. As the leading term of the error for the composite trapezoial rule, on the $i^{th}$ interval is
$$-\frac{1}{2M^2}\left(\frac{i}{N} - \frac{i-1}{N}\right)^2\left((a \circ \tilde \gamma)'(i/N)-(a \circ \tilde \gamma)'((i-1)/N)\right) \\ =-\frac{1}{2M^2N^2}\left((a \circ \tilde \gamma)'(i/N)-(a \circ \tilde \gamma)'((i-1)/N)\right).$$
Since
$$(a \circ \tilde \gamma)'(x)=a'(\tilde \gamma(x)) \cdot \tilde \gamma'(x)$$
it follows that
$$(a \circ \tilde \gamma)'(i/N)-(a \circ \tilde \gamma)'((i-1)/N) \\= a'(\tilde \gamma(i/N)) \cdot \tilde \gamma'(i/N) - a'(\tilde \gamma((i-1)/N)) \cdot \tilde \gamma'((i-1)/N)\\= a'(\mathbf x_i) \cdot \tilde \gamma'(i/N) - a'(\mathbf x_{i-1}) \cdot \tilde \gamma'((i-1)/N) \\=a'(\mathbf x_i) \cdot N(\mathbf x_{i+1} - \mathbf x_{i}) - a'(\mathbf x_{i-1}) \cdot N(\mathbf x_i - \mathbf x_{i-1}) \\= \left(a'(\mathbf x_i) - a'(\mathbf x_{i-1})\right) \cdot N(\mathbf x_i - \mathbf x_{i-1}).$$
There error therefore becomes
$$-\frac{1}{2M^2N^2}\left(a'(\mathbf x_i) - a'(\mathbf x_{i-1})\right) \cdot N(\mathbf x_i - \mathbf x_{i-1}).$$
The length is therefore
$$L[\tilde \gamma] = \sum_{i=1}^N N\|\mathbf x_i - \mathbf x_{i-1}\|T\left(M,\frac{i-1}{N},\frac{i}{N},a \circ \tilde \gamma \right) + O(N/M^2)$$

$$\bar{y}(t) = \mathbf{x_{i-1}} + t(\mathbf{x}_i - \mathbf{x}_{i-1}) \, \mathrm{in}\, t \in [0, 1)$$

In [1]:
import numpy as np
from typing import Callable

In [2]:
def compute_curve_length(
    xs: np.ndarray,
    metric_fn: Callable[[np.ndarray], np.ndarray],
    *,
    n_trapz_sample_points: int = 10,
) -> float:
    """
    Args:
        xs: points representing the piece-wise linear curve, an array of shape :math:`(N, D)` where 
            :math:`N - 1` is the number of line segments and :math:`D` is the dimension of the problem space. 
        metric_fn: a callable that takes in an array of :math:`D`-dimensional points of shape :math:`(\ldots, D)`
            and returns the metric evaluated at each point to produce an array of shape :math:`(\ldots)`.
        n_trapz_sample_points: The number of points that each line segment is split into
            which the metric_fn is evaluated at. This is used only within the evaluation of the 
            trapezoidal rule. Increasing this number will increase the precision of the approximation.
            
    Returns:
        Approximated length of the curve specified by ``xs`` and ``metric_fn`` using the trapezoidal rule. 
        More specifically the approximation of the length of the curve ``xs`` in the isotropic Riemannian manifold 
        characterized by ``metric_fn``.
        
    See Also:
        This function uses :py:func:`numpy.trapz` to perform the approximate integral along the curve.
    """
    
    diffs = xs[1:] - xs[:-1]  # (N - 1, D)
    
    trapz_sample_points = np.linspace(0, 1, n_trapz_sample_points)  # (M,)
    sample_points = (
        diffs[..., np.newaxis, :] *
        trapz_sample_points[:, np.newaxis] +
        xs[:-1, ..., np.newaxis, :]
    )  # (N - 1, M, D)

    metric_sample_values = metric_fn(sample_points)  # (N - 1, M)
    norms = np.linalg.norm(diffs, axis=-1)  # (N - 1)
    metric_coefficient_approximation = np.trapz(metric_sample_values, dx=(1 / (n_trapz_sample_points - 1)), axis=-1)  # (N - 1, M)
    line_segment_lengths = (norms * metric_coefficient_approximation)
    return line_segment_lengths.sum()

In [3]:
# PROBLEM SET UP
n_global_curve_points = 20  # N
n_trapz_sample_points = 9  # M
dimensionality = 5

# This is the tent function
def metric_fn(xs: np.ndarray, beta=0.6, weights=None):
    if weights is None:
        weights = np.ones(xs.shape[-1])
    return np.exp(-beta * (xs @ weights))

In [4]:
# Example 1: Straight line
xs = np.linspace(-2, 5, n_global_curve_points)
xs = np.stack([xs] + [np.zeros(n_global_curve_points)] * (dimensionality - 1), axis=-1)
compute_curve_length(xs, metric_fn, n_trapz_sample_points=n_trapz_sample_points)

5.45089654598387

In [5]:
# Example 2: Random line
np.random.seed(42)
xs = np.random.randn(n_global_curve_points, dimensionality)
compute_curve_length(xs, metric_fn, n_trapz_sample_points=n_trapz_sample_points)

102.54508034947423